## Settings

In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
import logging
import torch

In [4]:
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
project_root

'/Users/meina/Github/meina-t/matching_with_dl'

In [5]:
from model.model import MatchingNet
from model.params import HParams
from model.data import Data
from model.loss.strategy_proofness import compute_spv
from model.loss.stability import compute_sv
from model.loss.efficiency import compute_ev
from model.utils import da_with_t

In [14]:
cfg = HParams(
    num_agents=3,
    num_hidden_nodes=64,
    batch_size=128,
    epochs=2000,
    corr = 0,
    device='mps',
    lr=0.05,
    )
device = 'mps'

data = Data(cfg)

model = MatchingNet(cfg)
model.load_state_dict(torch.load(f'{project_root}/model/model_0120.pth'))
model.to(device)
model.eval()

/var/folders/3s/sh8zstl54dl_t2y55d7th57m0000gn/T/ipykernel_64191/3490232453.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{project_

MatchingNet(
  (layers): Sequential(
    (0): Linear(in_features=18, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=64, out_features=9, bias=True)
  )
)

## Examine

In [22]:
# この値を変える
p = torch.tensor([[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]],  dtype=torch.float32).to(device)
q = torch.tensor([[[1.0, 0.0, 0.0], [0.0,1.0, 0.0], [0.0, 0.0, 1.0]]],  dtype=torch.float32).to(device)

In [23]:
r = model(p, q)
r

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]], device='mps:0', grad_fn=<DivBackward0>)

In [24]:
da_r = da_with_t(p, q)
da_r

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]], device='mps:0')

In [25]:
spv = compute_spv(cfg, model, r, p, q).sum()
sv = compute_sv(cfg,r, p, q).sum()
ev = compute_ev(cfg, r, p, q, data).sum()
print(f'spv: {spv}, sv: {sv}, ev: {ev}')

spv: 1.2271823379705893e-06, sv: 0.0, ev: 0.0


In [26]:
da_spv = compute_spv(cfg, da_with_t, da_r, p, q).sum()
da_sv = compute_sv(cfg, da_r, p, q).sum()
da_ev = compute_ev(cfg, da_r, p, q, data).sum()
print(f'da_spv: {da_spv}, da_sv: {da_sv}, da_ev: {da_ev}')

da_spv: 0.0, da_sv: 0.0, da_ev: 0.0
